<a href="https://colab.research.google.com/github/iharashuuji/SFT_MINIMUM/blob/main/Deepseek%E7%B3%BB%E6%8E%A8%E8%AB%96.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 諸々のライブラリ準備等

In [ ]:
# Colab で安定する組み合わせをインストール
!pip install transformers==4.41.2 peft==0.10.0 datasets==2.19.0 accelerate==0.29.3 trl==0.8.6
!pip install -U bitsandbytes

In [ ]:
!pip install -U huggingface_hub

In [ ]:
!rm -f /root/.cache/huggingface/token
!rm -f /root/.cache/huggingface/stored_tokens

In [ ]:
import os
os.environ["HUGGINGFACE_TOKEN"] = ""
!huggingface-cli login

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGrained).
The token `Google_colab` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-

In [ ]:
import wandb
os.environ["WANDB_TOKEN"] = ""
!wandb login
wandb.init(project="Deepseek-7b-samples")

In [ ]:
# import transformers
# import trl
# import peft
# import datasets
# import accelerate

# print("Transformers version:", transformers.__version__)
# print("TRL version:", trl.__version__)
# print("PEFT version:", peft.__version__)
# print("Datasets version:", datasets.__version__)
# print("Accelerate version:", accelerate.__version__)

Transformers version: 4.41.2
TRL version: 0.8.6
PEFT version: 0.10.0
Datasets version: 2.19.0
Accelerate version: 0.29.3


In [ ]:
# from huggingface_hub import snapshot_download
# model_name = "deepseek-ai/deepseek-llm-7b-base"
# !mkdir /content/drive/MyDrive/LLM/model/{model_name}
# save_dir = f"/content/drive/MyDrive/LLM/model/{model_name}"  # 好きなパスを指定

# local_path = snapshot_download(
#     repo_id=model_name,
#     revision="main",
#     cache_dir=save_dir  # ← ここで保存先を指定
# )
# print("モデルをダウンロードしました:", local_path)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

README.md: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/584 [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.97G [00:00<?, ?B/s]

...

.gitattributes: 0.00B [00:00, ?B/s]

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.85G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/792 [00:00<?, ?B/s]

モデルをダウンロードしました: /content/drive/MyDrive/LLM/model/deepseek-7b/models--deepseek-ai--deepseek-llm-7b-base/snapshots/7683fea62db869066ddaff6a41d032262c490d4f


In [ ]:
# !ls /content/drive/MyDrive/LLM/model/deepseek-7b/models--deepseek-ai--deepseek-llm-7b-base/snapshots/7683fea62db869066ddaff6a41d032262c490d4f

config.json			  pytorch_model.bin.index.json
generation_config.json		  README.md
pytorch_model-00001-of-00002.bin  tokenizer_config.json
pytorch_model-00002-of-00002.bin  tokenizer.json


# モデル学習SFT

In [ ]:
# # Deepmathを使用する
# from hugginface_hub import datasets
# from datasets import load_dataset

# # Login using e.g. `huggingface-cli login` to access this dataset
# ds = load_dataset("zwhe99/DeepMath-103K")

In [ ]:
# データ用意
sample_data = [
    {"instruction": "Integrate x^2 dx", "input": "", "output": "x^3 / 3 + C"},
    {"instruction": "Derivative of sin(x)", "input": "", "output": "cos(x)"},
    {"instruction": "Solve equation: 2x + 3 = 7", "input": "", "output": "x = 2"},
    {"instruction": "Simplify: (x^2 * x^3)", "input": "", "output": "x^5"},
    {"instruction": "Integrate 1/x dx", "input": "", "output": "ln|x| + C"},
    {"instruction": "Derivative of e^x", "input": "", "output": "e^x"},
    {"instruction": "Solve equation: x^2 - 4 = 0", "input": "", "output": "x = 2 or x = -2"},
    {"instruction": "Simplify: (x^3)^2", "input": "", "output": "x^6"},
    {"instruction": "Integrate cos(x) dx", "input": "", "output": "sin(x) + C"},
    {"instruction": "Derivative of ln(x)", "input": "", "output": "1/x"},
]
# 変換後のデータを入れるための、空のリストを準備
new_formatted_data = []

# sample_dataリストの中身を、itemという変数に一つずつ入れてループ処理
for item in sample_data:
    # -------------------------------------------------------------
    # ここで、instructionの内容に応じた「思考プロセス」を組み立てる
    # (今回はシンプルな例として、固定の文章を入れます)
    # -------------------------------------------------------------
    instruction_text = item["instruction"]
    final_answer = item["output"]

    # instructionの内容から、簡単な思考プロセスを自動生成する
    if "Integrate" in instruction_text:
        thought_process = f"This is an integration problem. The goal is to find the antiderivative of the function given in the instruction."
    elif "Derivative" in instruction_text:
        thought_process = f"This is a differentiation problem. The goal is to find the derivative of the function."
    else:
        thought_process = "This is a math problem. I need to follow the instruction to find the answer."

    # 新しいoutputの文字列を f-string で組み立てる
    new_output = f"<Think>{thought_process}</Think><Answer>{final_answer}</Answer>"

    # 新しい形式の辞書を作成して、リストに追加する
    new_formatted_data.append({
        "instruction": instruction_text,
        "input": item["input"],
        "output": new_output
    })

# 結果を確認するために、最初の1件を表示
import json
print(json.dumps(new_formatted_data[0], indent=2, ensure_ascii=False))

In [ ]:
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from trl import SFTTrainer
from peft import LoraConfig

local_path = "/content/drive/MyDrive/LLM/model/deepseek-7b/models--deepseek-ai--deepseek-llm-7b-base/snapshots/7683fea62db869066ddaff6a41d032262c490d4f"

# これで正常にロードされる
tokenizer = AutoTokenizer.from_pretrained(local_path)

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

model = AutoModelForCausalLM.from_pretrained(
    local_path,
    device_map="auto",
    quantization_config=quant_config
)
# 10件サンプル


# 3. LoRA 設定
peft_config = LoraConfig(
    r=8,
    lora_alpha=8,
    target_modules=["q_proj", "v_proj"],  # 注意: モデルによって要調整
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# from transformers import AutoTokenizer
# from datasets import Dataset

# dataset = Dataset.from_list(new_formatted_data)
# if tokenizer.pad_token is None:
#     tokenizer.add_special_tokens({'pad_token': '[PAD]'})
#     model.resize_token_embeddings(len(tokenizer))


# # 2. tokenize 関数
# def tokenize_fn(example):
#     return tokenizer(
#         example['instruction'],
#         truncation=True,
#         padding="max_length",
#         max_length=128
#     )

# # 3. 実行
# tokenized_dataset = dataset.map(tokenize_fn, batched=True)

TypeError: list indices must be integers or slices, not str

In [ ]:
# 1. まず、関数を「定義」する
#    この時点では、`formatting_func`は関数オブジェクトです
def formatting_func(example_batch):
    texts = []
    for i in range(len(example_batch["instruction"])):
        text = f"Instruction: {example_batch['instruction'][i]}\n"
        if example_batch["input"][i]:
            text += f"Input: {example_batch['input'][i]}\n"
        text += f"Answer: {example_batch['output'][i]}"
        texts.append(text)
    return texts

In [ ]:
dataset = Dataset.from_list(new_formatted_data)
print(dataset)

In [ ]:
from transformers import TrainingArguments
from trl import SFTTrainer
train_dataset_obj = Dataset.from_list(new_formatted_data)



training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    num_train_epochs=2,
    learning_rate=5e-5,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch",
    push_to_hub=False,
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset, #tokenized_dataset,
    peft_config=peft_config,
    args=training_args,
    formatting_func=formatting_func,
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.12/dist-packages/trl/trainer/sft_trainer.py:246: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/2 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [ ]:
# デバッグをしやすいように書いて置く。
%env CUDA_LAUNCH_BLOCKING=1

env: CUDA_LAUNCH_BLOCKING=1


In [ ]:
train_output = trainer.train()

# 学習結果のサマリを確認
print(train_output)

Step,Training Loss


/usr/local/lib/python3.12/dist-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in /content/drive/MyDrive/LLM/model/deepseek-7b/models--deepseek-ai--deepseek-llm-7b-base/snapshots/7683fea62db869066ddaff6a41d032262c490d4f - will assume that the vocabulary was not modified.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in /content/drive/MyDrive/LLM/model/deepseek-7b/models--deepseek-ai--deepseek-llm-7b-base/snapshots/7683fea62db869066ddaff6a41d032262c490d4f - will assume that the vocabulary was not modified.
  warnings.warn(


TrainOutput(global_step=2, training_loss=1.3456859588623047, metrics={'train_runtime': 1.67, 'train_samples_per_second': 2.395, 'train_steps_per_second': 1.198, 'total_flos': 8417348222976.0, 'train_loss': 1.3456859588623047, 'epoch': 2.0})


In [ ]:
merged_model = trainer.model.merge_and_unload()

/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/bnb.py:325: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [ ]:
# model = model.to("cpu")
# model.resize_token_embeddings(len(tokenizer))
# model = model.to("cuda")

# # PAD トークンが存在するか確認
# print("PAD token:", tokenizer.pad_token, "PAD id:", tokenizer.pad_token_id)
# print("Vocab size:", len(tokenizer))

# # モデルに合わせて埋め込みをリサイズ
# model.resize_token_embeddings(len(tokenizer))

In [ ]:
# from transformers import AutoTokenizer

# save_dir = "./my-finetuned-model"
# trainer.model.save_pretrained(save_dir)
# tokenizer.save_pretrained(save_dir)

In [ ]:
# from huggingface_hub import HfApi, HfFolder, Repository

# # ユーザー名/組織名 + リポジトリ名
# repo_id = "Kashiwahorito/deepseek-1.3b-sft-model"

# # モデルをHubにプッシュ
# from huggingface_hub import upload_folder
# upload_folder(
#     repo_id=repo_id,
#     folder_path=save_dir,
#     commit_message="Upload finetuned model"
# )

In [ ]:
# print("pad:", tokenizer.pad_token, tokenizer.pad_token_id)
# print("eos:", tokenizer.eos_token, tokenizer.eos_token_id)
# print("bos:", tokenizer.bos_token, tokenizer.bos_token_id)
# print("unk:", tokenizer.unk_token, tokenizer.unk_token_id)
# print("vocab size:", len(tokenizer))

pad: <｜end▁of▁sentence｜> 32014
eos: <｜end▁of▁sentence｜> 32014
bos: <｜begin▁of▁sentence｜> 32013
unk: None None
vocab size: 32022


# GRPO

In [ ]:
# GRPO用のデータを用意する
# 使用できる関数は、


# 推論

In [ ]:
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
"""
もう少し大きなモデルで実験を行う。
"""
 # hf_hub_download などで置いたフォルダ
# offload_folder = "/content/offload"

# 推論用パイプライン
pipe = pipeline(
    "text-generation",
    model= merged_model,
    tokenizer=tokenizer,
    device_map="auto",
    do_sample=True
)

prompt = """
Solve the integral of x^2 from 0 to 1.
Please think output your reasoning within <Think>...</Think> tags,
then give the final answer within <Answer>...</Answer> tags.
"""

prompt_step_by_step = """Solve the integral of x^2 from 0 to 1.
Please think step by step and output your reasoning within <Think>...</Think> tags,
then give the final answer within <Answer>...</Answer> tags."""

output = pipe(prompt, max_new_tokens=200, tempreture=0.7, top_p~0.9)
outputs_step_by_step = pipe(prompt_step_by_step, max_new_tokens=200, temperature=0.7, top_p=0.9)
print(f"outputs_step_by_step: {outputs_step_by_step[0]["generated_text"]}")
print(f"outputs: {outputs[0]["generated_text"]}")

Solve the integral of x^2 from 0 to 1.
Please think step by step and output your reasoning within <Think>...</Think> tags, 
then give the final answer within <Answer>...</Answer> tags.


A: We want to integrate $\int_0^1 x^2 dx$.
We can rewrite this as $\int_0^1 \frac{1}{2} x^2 dx$ by substituting $x^2$ with $x$ in the integral.
Now we can integrate using the power rule: $\int_0^1 x^2 dx = \frac{x^3}{3} |_{0}^1 = \frac{1}{3} - \frac{0}{3} = \frac{1}{3}$.

A: The answer is $\displaystyle \frac{1}{3}$. 
We can start by changing the limits of integration to be $0$ and $1$ so that we can use the power rule: 
$$\int_0^1 x^2 dx = \frac{x^3}{3} |_{0}^1 = \frac{1}{3}$$




In [ ]:
import re

text = outputs[0]["generated_text"]

# タグ抽出
think = re.search(r"<Think>(.*?)</Think>", text, re.DOTALL)
answer = re.search(r"<Answer>(.*?)</Answer>", text, re.DOTALL)

think_text = think.group(1).strip() if think else ""
answer_text = answer.group(1).strip() if answer else ""

# 数値として採点
try:
    answer_value = float(eval(answer_text))  # 1/3 を float に変換
except:
    answer_value = None

correct_value = 1/3
score = 1.0 if answer_value is not None and abs(answer_value - correct_value) < 1e-2 else 0.0

print("Think:", think_text)
print("Answer:", answer_text)
print("Score:", score)

Think: ...
Answer: ...
Score: 0.0


# 実験結果

**実験結果Deepseek-1.3B**<br>
・訓練なしモデルの回答は以下<br>
Device set to use cuda:0
Solve the integral of x^2 from 0 to 1.
Please think step by step and output your reasoning within <Think>...</Think> tags,
then give the final answer within <Answer>...</Answer> tags.
```
<Think><Answer></Answer></Think>
```
<br>
・訓練済みモデルの回答は以下<br>
Device set to use cuda:0 /usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None warnings.warn( Solve the integral of x^2 from 0 to 1. Please think step by step and output your reasoning within

```
<Think>...</Think> tags, then give the final answer within <Answer>...</Answer> tags. public void init(11. // 2); } .with_100000000000000000000000000000000000, 1152001, } public string(5 if (1) return (2, public static final int[0000000122410000000000000001; import { # 2, } } } public string) import { public String user.util.1000000001600000.set[17.util.
```


# プロンプト実験

**Step by Step**で考えさせた解答を以下に記載をしておく
```
Solve the integral of x^2 from 0 to 1.
Please think step by step and output your reasoning within <Think>...</Think> tags,
then give the final answer within <Answer>...</Answer> tags.


A: We want to integrate $\int_0^1 x^2 dx$.
We can rewrite this as $\int_0^1 \frac{1}{2} x^2 dx$ by substituting $x^2$ with $x$ in the integral.
Now we can integrate using the power rule: $\int_0^1 x^2 dx = \frac{x^3}{3} |_{0}^1 = \frac{1}{3} - \frac{0}{3} = \frac{1}{3}$.

A: The answer is $\displaystyle \frac{1}{3}$.
We can start by changing the limits of integration to be $0$ and $1$ so that we can use the power rule:
$$\int_0^1 x^2 dx = \frac{x^3}{3} |_{0}^1 = \frac{1}{3}$$
```